<table>
    <tr>
        <td><img src="./img/Macc.png" width="auto"/></td>
        <td>
            <table><tr>
            <h1 style="color:blue;text-align:center">Lógica para Ciencias de la Computación</h1></td>
            </tr></table>   
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Taller</p></tp>
            <tp><p style="font-size:150%;text-align:center">Equivalencia lógica</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivos <a class="anchor" id="inicio"></a>

Una vez tenemos nuestra implementación de las fórmulas de la lógica proposicional como clases en Python, podemos comenzar a hacer pruebas interesantes. En este notebook vamos a determinar si dos fórmulas son equivalentes.

# Secciones

1. [Equivalencia lógica.](#eq)
2. [Sustituciones.](#sus)
3. [Forma normal conjuntiva.](#fnc)

# Equivalencia lógica <a class="anchor" id="eq"></a>

([Volver al inicio](#inicio))

Sean $A$, $B$, fórmulas. La equivalencia entre $A$ y $B$ (denotada por $A\equiv B$) se define de la siguiente manera:

$$
A\equiv B  { si, y solamente si, }A.valor(I)=B.valor(I){ para toda interpretación }I
$$

Observe que ya habíamos usado la equivalencia anteriormente. Hicimos uso de ella para encontrar cuál era la prioridad de las operaciones booleanas en Python. En este caso revisamos las tablas de verdad de ambas variables para ver si las columnas eran todas iguales.

Vamos a operacionalizar esto en una función `chequear_equivalencia` en Python, la cual toma dos fórmulas (como árboles) como argumento. La función debe recorrer cada una de las interpretaciones posibles, y detener si no tienen el mismo valor. Si recorre todas sin detenerse, entonces son equivalentes.

Observe que tenemos acceso al archivo `Logica.py`, en el cual están implementadas las fórmulas y la función `inorder_to_tree`:

In [1]:
# De python
from copy import deepcopy
# Del notebook
from Logica import *

In [2]:
A = inorder_to_tree('--p')
I = {'p':True}
A.valor(I)

True

**Ejercicio 1:** 

Implemente la función `chequear_equivalencia`.

Verifique su implementación para corroborar las siguientes equivalencias:

* $(p \to q)\equiv(\neg p\vee q)$
* $\neg(p\wedge q)\equiv (\neg p\vee\neg q)$
* $\neg(p\vee q)\equiv (\neg p\wedge\neg q)$
* $\bigl(p\vee(q\wedge r)\bigr)\equiv\bigl((p\vee q)\wedge(p\vee r)\bigr)$
* $\bigl(p\wedge(q\vee r)\bigr)\equiv\bigl((p\wedge q)\vee(p\wedge r)\bigr)$

Corrobore también que las siguientes NO son equivalencias:

* $(p\to q)\not\equiv(q\to p)$
* $\bigl(p\vee(q\wedge r)\bigr)\not\equiv\bigl((p\wedge q)\vee(p\wedge r)\bigr)$

In [3]:
def chequear_equivalencia(A, B):
    letras = list(A.letras().union(B.letras()))
    num_letras = len(letras)
    interpretaciones = [dict(zip(letras, list(product([True, False], repeat=num_letras))))]
    for I in interpretaciones:
        if A.valor(I) != B.valor(I):
            return False
    return True

In [8]:
A1 = inorder_to_tree('(p>q)')
A2 = inorder_to_tree('(-pOq)')
print(chequear_equivalencia(A1, A2))

A1 = inorder_to_tree('-(pYq)')
A2 = inorder_to_tree('(-pO-q)')
print(chequear_equivalencia(A1, A2))

A1 = inorder_to_tree('-(pOq)')
A2 = inorder_to_tree('(-pY-q)')
print(chequear_equivalencia(A1, A2))

A1 = inorder_to_tree('(pO(qYr))')
A2 = inorder_to_tree('((pOq)Y(pOr))')
print(chequear_equivalencia(A1, A2))

A1 = inorder_to_tree('(pY(qOr))')
A2 = inorder_to_tree('((pYq)O(pYr))')
print(chequear_equivalencia(A1, A2))

print('---------------falsas----------')

A1 = inorder_to_tree('(p>q)')
A2 = inorder_to_tree('(q>p)')
print(chequear_equivalencia(A1, A2))

A1 = inorder_to_tree('(pO(qYr))')
A2 = inorder_to_tree('((pYq)O(pYr))')
print(chequear_equivalencia(A1, A2))


True
True
True
True
True
---------------falsas----------
False
False


---

# Sustituciones <a class="anchor" id="sus"></a>

([Volver al inicio](#inicio))

Recordemos que el siguiente es el pseudocódigo de sustituir una subfórmula $A$ de $B$ por otra fórmula $A'$:

```python
funcion sust(self, A, A'):
    Si A no está en self.subforms():
        retornar self
    Si no, si A es self:
        retornar A'
    Si no, si self es de tipo Negacion:
        retornar Negacion(self.subf.sust(A,A'))
    Si no, si self es de tipo Binario:
        retornar Binario(self.conectivo, 
                         self.left.sust(A,A'), 
                         self.right.sust(A,A'))
```

**Ejercicio 2:**

Implemente el método sust(A,A').

**Nota:** No requiere implementar el método `subforms`, toda vez que este ya está implementado en la clase `Formula` de la librería `Logica`. Observe que dicho método `subforms` devuelve una lista de strings. Tenga esto en cuenta al momento de hacer las comparaciones requeridas por los dos primeros casos del método `sust`.


In [16]:
B = inorder_to_tree("-(pYq)")
B.subforms()

['-(pYq)', 'q', '(pYq)', 'p']

In [25]:
def sust(self, A, C):
    if str(A) not in self.subforms():
        return self
    elif str(A) == str(self):
        return C
    elif type(self) == Negacion:
        return Negacion(self.subf.sust(A,C))
    elif type(self) == Binario:
        return Binario(self.conectivo, self.left.sust(A, C), self.right.sust(A, C))
         
setattr(Formula, "sust", sust)

In [27]:
p = Letra('(pYq)')
s = Letra('s')
print(B.sust(p, s))

-s


---

# Forma Normal Conjuntiva <a class="anchor" id="fnc"></a>

([Volver al inicio](#inicio))

Procedimiento para transformar una fórmula arbitraria $A$ en una fórmula $A'$ en forma normal conjuntiva, tal que $A\equiv A'$:

1. Eliminar `$\leftrightarrow$' y '$\to$'.
2. Eliminar dobles negaciones.
3. Si $\neg(B\wedge C)\in A.{subform}()$, reemplazarla por $\neg B\vee\neg C$.
4. Si $\neg(B\vee C)\in A.{subform}()$, reemplazarla por $\neg B\wedge\neg C$.
5. Eliminar dobles negaciones.
6. Si $B\vee (C\wedge D)\in A.{subform}()$, reemplazarla por $(B\vee C)\wedge (B\vee D)$.


In [28]:
from copy import deepcopy

def eliminar_doble_imp(self):
    if type(self) == Letra:
        return self
    elif type(self) == Negacion:
        return Negacion(self.subf.eliminar_doble_imp())
    elif type(self) == Binario:
        if self.conectivo == '=':
            return Binario('Y',
                           Binario('O', 
                               Negacion(self.left.eliminar_doble_imp()),
                               self.right.eliminar_doble_imp(),
                              ),
                           Binario('O', 
                               Negacion(self.right.eliminar_doble_imp()),
                               self.left.eliminar_doble_imp(),
                              ))
        else:
            return Binario(self.conectivo,
                       self.left.eliminar_doble_imp(),
                       self.right.eliminar_doble_imp()
                      )     

setattr(Formula,"eliminar_doble_imp", eliminar_doble_imp)
        
def eliminar_doble_negacion(self):
    if type(self) == Letra:
        return self
    elif type(self) == Negacion:
        if type(self.subf) == Negacion:
            return deepcopy(self.subf.subf.eliminar_doble_negacion())
        else:
            return Negacion(self.subf.eliminar_doble_negacion())
    elif type(self) == Binario:
        return Binario(self.conectivo,
                       self.left.eliminar_doble_negacion(),
                       self.right.eliminar_doble_negacion()) 

setattr(Formula,"eliminar_doble_negacion", eliminar_doble_negacion)

def cambiar_de_morgan_y(self):
    if type(self) == Letra:
        return self
    elif type(self) == Negacion:
        if type(self.subf) == Binario:
            if self.subf.conectivo == 'Y':
                return Binario('O', 
                               Negacion(self.subf.left.cambiar_de_morgan_y()),
                               Negacion(self.subf.right.cambiar_de_morgan_y())
                              )
            else:
                return Negacion(self.subf.cambiar_de_morgan_y()) 
        else:
            return Negacion(self.subf.cambiar_de_morgan_y())
    elif type(self) == Binario:
        return Binario(self.conectivo,
                       self.left.cambiar_de_morgan_y(),
                       self.right.cambiar_de_morgan_y()
                      ) 

setattr(Formula,"cambiar_de_morgan_y", cambiar_de_morgan_y)

def distribuir_o_en_y(self):
    if type(self) == Letra:
        return self
    elif type(self) == Negacion:
        return Negacion(self.subf.distribuir_o_en_y())
    elif type(self) == Binario:
        if self.conectivo == 'O':
            # print('O')
            if type(self.right) == Binario:
                # print('right binario')
                if self.right.conectivo == 'Y': # B O (C Y D)
                    # print('right Y')
                    B = self.left.distribuir_o_en_y()
                    C = self.right.left.distribuir_o_en_y()
                    D = self.right.right.distribuir_o_en_y()
                    return Binario('Y',
                                   Binario('O', B, C),
                                   Binario('O', B, D)
                                  )
            if type(self.left) == Binario:
                # print('left binario')
                if self.left.conectivo == 'Y': # (B Y C) O D
                    # print('left Y')
                    B = self.left.left.distribuir_o_en_y()
                    C = self.left.right.distribuir_o_en_y()
                    D = self.right.distribuir_o_en_y()
                    return Binario('Y',
                                   Binario('O', B, D),
                                   Binario('O', C, D)
                                  )
    return Binario(self.conectivo,
                   self.left.distribuir_o_en_y(),
                   self.right.distribuir_o_en_y()
                  )

setattr(Formula,"distribuir_o_en_y", distribuir_o_en_y)


In [29]:
A = inorder_to_tree("--(-(r>q)=(--pYq))")
B = A.eliminar_doble_imp()
print(f'Al eliminar las dobles implicaciones de \n{str(A)}\nse obtiene:\n{str(B)}')
C = B.eliminar_doble_negacion()
print(f'Al eliminar las dobles negaciones se obtiene:\n{str(C)}')
D = C.cambiar_de_morgan_y()
print(f'Al reemplazar -(AYB) por -AO-B se obtiene:\n{str(D)}')


Al eliminar las dobles implicaciones de 
--(-(r>q)=(--pYq))
se obtiene:
--((--(r>q)O(--pYq))Y(-(--pYq)O-(r>q)))
Al eliminar las dobles negaciones se obtiene:
(((r>q)O(pYq))Y(-(pYq)O-(r>q)))
Al reemplazar -(AYB) por -AO-B se obtiene:
(((r>q)O(pYq))Y((-pO-q)O-(r>q)))


**Ejercicio 3:**

Observe que hemos omitido las dos funciones siguientes, las cuales usted debe completar:

In [35]:
def eliminar_imp(self):
    if type(self) == Letra:
        return self
    elif type(self) == Negacion:
        return Negacion(self.subf.eliminar_imp())
    elif type(self) == Binario:
        if self.conectivo == '>':
            return Binario('O', Negacion(self.left), self.right)
        else:
            return Binario(self.conectivo, self.left.eliminar_imp(), self.right.eliminar_imp())

setattr(Formula,"eliminar_imp", eliminar_imp)

def cambiar_de_morgan_o(self):
    if type(self) == Letra:
        return self
    elif type(self) == Negacion:
        if type(self.subf) == Binario and self.subf.conectivo == 'O':
            return Binario('Y', Negacion(self.subf.left), Negacion(self.subf.right))
        else:
            return Negacion(self.subf)
    elif type(self) == Binario:
        return Binario(self.conectivo, self.left.cambiar_de_morgan_o(), self.right.cambiar_de_morgan_o())

setattr(Formula,"cambiar_de_morgan_o", cambiar_de_morgan_o)

Complete los dos métodos de modificación anteriores. Al probar su código con la siguiente celda, la respuesta debe ser:

```
Fórmula inicial:
((pOq)>(rY-s))
Al eliminar las implicaciones se obtiene:
(-(pOq)O(rY-s))
Al reemplazar -(AOB) por -AY-B se obtiene:
((-pY-q)O(rY-s))
Al distribuir O en Y se obtiene:
(((-pY-q)Or)Y((-pY-q)O-s))
Al distribuir O en Y se obtiene:
(((-pOr)Y(-qOr))Y((-pO-s)Y(-qO-s)))
```

In [36]:
A = inorder_to_tree('((pOq)>(rY-s))')
B = A.eliminar_imp()
print(f'Al eliminar las implicaciones de \n{str(A)}\nse obtiene:\n{str(B)}')
C = B.cambiar_de_morgan_o()
print(f'Al reemplazar -(AYB) por -AO-B se obtiene:\n{str(C)}')
D = C.distribuir_o_en_y()
print(f'Al distribuir O en Y se obtiene:\n{str(D)}')
E = D.distribuir_o_en_y()
print(f'Al distribuir O en Y se obtiene:\n{str(E)}')

Al eliminar las implicaciones de 
((pOq)>(rY-s))
se obtiene:
(-(pOq)O(rY-s))
Al reemplazar -(AYB) por -AO-B se obtiene:
((-pY-q)O(rY-s))
Al distribuir O en Y se obtiene:
(((-pY-q)Or)Y((-pY-q)O-s))
Al distribuir O en Y se obtiene:
(((-pOr)Y(-qOr))Y((-pO-s)Y(-qO-s)))


---